In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import gc
import nltk
nltk.download('stopwords')
import math

from smart_open import open
from nltk.corpus import stopwords
import sklearn
from sklearn.metrics import classification_report
from keras.layers import Dropout
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras import layers


from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
#Text
class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True) 


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")    


In [25]:
#Audio

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)  


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")
#Audio

class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)  


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")


In [26]:
#Text + Audio 
class Highway(layers.Layer):

  def __init__(self):
    super(Highway, self).__init__()

  def build(self, input_shape):
    n_sentences = input_shape[1]
    n_features = input_shape[2]
    carry_bias = keras.initializers.Constant(value=-1.0)
    random_dist = keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

    carry_bias_2 = keras.initializers.Constant(value= 0.1)

    self.W_T = self.add_weight(shape=(n_features, n_features),initializer = random_dist,trainable=True)
    self.b_T = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias, trainable=True)
    # self.W_T = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1),trainable=True)
    # self.b_T = tf.Variable(tf.constant(-1.0, shape=[n_data_points, n_sentences, n_features]),trainable=True)
   
    self.W = self.add_weight(shape=( n_features, n_features),initializer = random_dist, trainable=True)
    self.b = self.add_weight(shape=( n_sentences, n_features),initializer = carry_bias_2, trainable=True)
    # self.W = tf.Variable(tf.random.truncated_normal([n_data_points,n_features, n_features], stddev=0.1), trainable=True)
    # self.b = tf.Variable(tf.constant(0.1, shape=[n_data_points, n_sentences, n_features]), trainable=True)  


  def call(self, inputs):
    x = inputs
    T = tf.sigmoid(tf.matmul(x, self.W_T) + self.b_T, name="transform_gate")
    H = tf.nn.relu(tf.matmul(x, self.W) + self.b, name="activation")
    C = tf.subtract(1.0, T, name="carry_gate")
    
    return tf.add(tf.multiply(H, T), tf.multiply(x, C), "y")

class text_audio:
# first input model
  input1 = Input(shape=(250,74), name = 'Audio_input')
  # highway6 = Highway()(highway5)
  dense1 = Dense(74)(input1)

  input3 = Input(shape = (250,5100), name = 'Text_input')
  dense4 = Dense(1000)(input3)
  dense5 = Dense(500)(dense4)
  dense6 = Dense(250)(dense5)
  dense3 = Dense(74)(dense6)
  # merge input models
  merge = concatenate([dense1,dense3], axis = 1)
  # interpretation model
  lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(merge)
  output = Dense(1, activation='sigmoid')(lstm)
  model = Model(inputs=[input1, input3], outputs=output)
  # summarize layers
  # print(model.summary())
  # # plot graph
  # plot_model(model)
  optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)

  def run_model(self):
    self.model.compile(optimizer=self.optimizer, loss='binary_crossentropy')
    return self.model    

In [27]:
#MAKE DATASET 


dev_location = "dev_data"
test_location = "test_data"
train_location = "train_data"

devData = np.array(pd.read_csv('/content/drive/My Drive/Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
testData = np.array(pd.read_csv('/content/drive/My Drive/Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
trainData = np.array(pd.read_csv('/content/drive/My Drive/Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


dataset = np.concatenate((devData, np.concatenate((testData, trainData))))

gc.collect()      
max_num_words = 17
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Dataset/GoogleNews-vectors-negative300.bin.gz', binary=True)
stop_words = set(stopwords.words('english'))



def checkDataPointExistence(patientID, split):
  for i in split:
    if(patientID == i[0]):
      return True
  return False
def getData(patientID, location):
  # print("PatientID: " + str(int(patientID)))
  retData = [int(patientID)]
  textD = getTextData(patientID, location)
  audioD = getAudioData(patientID, location, textD)
  # patientD = np.concatenate((textD, audioD), axis = 1)
  # print("Final Patient Data: " + str(patientD.shape))
  return textD,audioD

def getTextData(patientID, location):
  fileName = "/content/drive/My Drive/Dataset/"+ str(location) + "/" + str(int(patientID)) + "_TRANSCRIPT.csv"
  file = np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))

  # Remove All Utterences By Ellie:
  for i in range(len(file)):
    if(file[i][2] != 'Participant'):
      np.delete(file, i)
      i-=1

  # Remove Speaker Columnn
  file = np.delete(file, 2, 1)
  
  # Convert Text Into Word Vectors:
  w2vs = np.zeros((1, max_num_words*300))
  for i in range(len(file)):
    sentence = file[i][2]
    w2v = returnWordToVec(sentence)
    w2vs = np.concatenate((w2vs, w2v), axis = 0)
  w2vs = np.delete(w2vs, 0, 0)  

  # Delete Sentences and Replace With W2Vs
  file = np.delete(file, 2, 1)
  file = np.concatenate((file, w2vs), axis = 1)
  return file
def remove_StopWords(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def returnWordToVec(sentence):
  global max_num_words, stop_words, model
  sentence = str(sentence).split(" ")
  sentence = remove_StopWords(sentence)
  index_word = 0
  wordMatrix = np.zeros(max_num_words*300)
  for j in range(min(max_num_words, len(sentence))):
    try:
      word = sentence[j]
      if(word[0] == '<'):
        if(word.find('>')!=-1):
          word = word[1:-1]
        else:
          word = word[1:]
      else:
        if(word.find('>')!=-1):
          word = word[0:-1]
      ss = np.array(model[word])
      wordMatrix[index_word*300:(index_word+1)*300] = ss
      index_word+=1
    except Exception as e:
      continue
  wordMatrix = np.array(wordMatrix.reshape(1,-1))
  return wordMatrix
def audioDataHelper(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    return X
    
def getAudioData(patientID, location, textD):
  fileName = "/content/drive/My Drive/Dataset/"+ str(location) + "/" + str(int(patientID)) + "_COVAREP.csv"
  data = pd.read_csv(fileName,header = None)
  data = data.iloc[:,:].values
  data = audioDataHelper(data)
  # print("Audio Raw Data:" + str(data.shape))
  sentenceDatas = []
  for sentence in textD:
    sentenceStartime = sentence[0]
    sentenceEndTime = sentence[1]
    startIndex = math.floor(sentenceStartime/0.01)
    endIndex = math.ceil(sentenceEndTime/0.01)
    # print("Start Time: " + str(startIndex))
    # print("End Time: " + str(endIndex))
    sentenceData = data[startIndex: endIndex]
    sentenceData = np.average(sentenceData, axis = 0)
    # This might be a possible error
    sentenceData = np.array(sentenceData.reshape(1, -1))
    sentenceDatas.append(sentenceData)
  
  sentenceDatas = np.array(sentenceDatas)
  sentenceDatas = np.reshape(sentenceDatas, (textD.shape[0],-1))
  # print("Audio Final Data:" + str(sentenceDatas.shape))

  return sentenceDatas
# Xtrain = []
Ytrain = []
# Xtest = []
Ytest = []


audio_train = []
text_train = []

audio_test = []
text_test = []


for datapoint in dataset:
  # print(datapoint[0])
  if(checkDataPointExistence(datapoint[0], devData)):

    # Data Point in Dev Set
    text,audio = getData(datapoint[0], dev_location)
    audio_train.append(audio)
    text_train.append(text)
    # Xtest.append(data)
    Ytrain.append(datapoint[1])
    # print(data)
  elif(checkDataPointExistence(datapoint[0], testData)):
    # Data Point in Test Set
    text,audio = getData(datapoint[0], test_location)
    audio_test.append(audio)
    text_test.append(text)
    # Xtest.append(data)
    Ytest.append(datapoint[1])
  elif(checkDataPointExistence(datapoint[0], trainData)):
    # Data Point in Train Set
    text,audio = getData(datapoint[0], train_location)
    audio_train.append(audio)
    text_train.append(text)
    # Xtest.append(data)
    Ytrain.append(datapoint[1])

def refactor(arr, size):
  arrsize = arr.shape[0]
  temp = np.zeros((size, arr.shape[1]))
  for i in range(min(len(arr), size)):
    temp[i] = arr[i]
  return temp
numberOfSentences = 250

devData = []
trainData = []
testData = []
gc.collect()

for i in range(len(audio_train)):
  audio_train[i] = refactor(audio_train[i], numberOfSentences)
  text_train[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtrain[i].shape)

# for i in range(len(audio_dev)):
#   audio_dev[i] = refactor(audio_dev[i], numberOfSentences)
#   text_dev[i] = refactor(text_dev[i], numberOfSentences)
#   # print(Xtrain[i].shape)

for i in range(len(audio_test)):
  audio_test[i] = refactor(audio_train[i], numberOfSentences)
  text_test[i] = refactor(text_train[i], numberOfSentences)
  # print(Xtest[i].shape)
audio_test = np.array(audio_test)
text_test = np.array(text_test)
text_test = text_test[:,:,2:]

audio_train = np.array(audio_train)
text_train = np.array(text_train)
text_train = text_train[:,:,2:]

# audio_dev = np.array(audio_dev)
# text_dev = np.array(text_dev)
# text_dev = text_dev[:,:,2:]

dataset = []
gc.collect()

print(audio_test.shape,text_test.shape)
print(audio_train.shape,text_train.shape)
# print(audio_dev.shape,video_dev.shape,text_dev.shape)

Ytrain = np.array(Ytrain)
Ytest = np.array(Ytest)


import sklearn
from sklearn import preprocessing

def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train

audio_train = np.nan_to_num(audio_train)
text_train = np.nan_to_num(text_train)

audio_train, _ = upsample(audio_train,Ytrain)
text_train, Ytrain = upsample(text_train,Ytrain)

print(audio_train.shape)
print(text_train.shape)
print(Ytrain.shape)

for i in range(audio_train.shape[0]):
  audio_train[i] = sklearn.preprocessing.normalize(audio_train[i])
  text_train[i] = sklearn.preprocessing.normalize(text_train[i])


print(Ytest.shape)



audio_test = np.nan_to_num(audio_test)
text_test = np.nan_to_num(text_test)


for i in range(audio_test.shape[0]):
  audio_test[i] = sklearn.preprocessing.normalize(audio_test[i])
  text_test[i] = sklearn.preprocessing.normalize(text_test[i])

(16, 250, 74) (16, 250, 5100)
(50, 250, 74) (50, 250, 5100)
(88, 250, 74)
(88, 250, 5100)
(88,)
(16,)


In [28]:
def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  print("Y_pred: ", Y_pred.shape)
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

In [29]:
print("-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------")
input3 = Input(shape = (250,5100))
dense4 = Dense(1000)(input3)
dense5 = Dense(500)(dense4)
dense6 = Dense(250)(dense5)
# interpretation model
lstm = LSTM(128, dropout = 0.2, recurrent_dropout = 0.2)(dense6)
output = Dense(1, activation='sigmoid')(lstm)
model = Model(inputs=input3, outputs=output)
# summarize layers
# print(model.summary())
# plot graph
# print(plot_model(model, to_file='multiple_inputs.png'))
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')   


model.fit(text_train,Ytrain, epochs=50,validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='min',
    baseline=None, restore_best_weights=True), batch_size = 137)




from sklearn.metrics import classification_report
pred = model.predict(text_test)

print(classification_report(Ytest,Thresholding(pred,0.5)))
# print(classification_report(Ytest,Thresholding(pred,0.6)))
# print(classification_report(Ytest,Thresholding(pred,0.4)))
# print(classification_report(Ytest,Thresholding(pred,0.3)))
# print(classification_report(Ytest,Thresholding(pred,0.7)))

-------------------------------------- TEXT (W/o GATING) SENTENCE LEVEL-------------------------------------------------
Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 0.6920 - val_loss: 0.6846
Epoch 2/50
1/1 [==============================] - 3s 3s/step - loss: 0.6800 - val_loss: 0.6771
Epoch 3/50
1/1 [==============================] - 3s 3s/step - loss: 0.6677 - val_loss: 0.6695
Epoch 4/50
1/1 [==============================] - 3s 3s/step - loss: 0.6557 - val_loss: 0.6615
Epoch 5/50
1/1 [==============================] - 3s 3s/step - loss: 0.6436 - val_loss: 0.6529
Epoch 6/50
1/1 [==============================] - 3s 3s/step - loss: 0.6310 - val_loss: 0.6436
Epoch 7/50
1/1 [==============================] - 3s 3s/step - loss: 0.6179 - val_loss: 0.6338
Epoch 8/50
1/1 [==============================] - 3s 3s/step - loss: 0.6058 - val_loss: 0.6236
Epoch 9/50
1/1 [==============================] - 3s 3s/step - loss: 0.5930 - val_loss: 0.6131
Epoch 10/50
1/1 [=======

In [37]:
print("--------------------------------------AUDIO (W/o GATING) SENTENCE LEVEL-------------------------------------------------")
input3 = Input(shape = (250,74))
# dense4 = Dense(1000)(input3)
# dense5 = Dense(500)(dense4)
# dense6 = Dense(250)(dense5)
# interpretation model
lstm = LSTM(60, dropout = 0.2, recurrent_dropout = 0.2)(input3)
output = Dense(1, activation='sigmoid')(lstm)
model = Model(inputs=input3, outputs=output)
  # summarize layers
  # print(model.summary())
  # plot graph
  # print(plot_model(model, to_file='multiple_inputs.png'))
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy')

model.fit(audio_train,Ytrain, validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=30, verbose=0, mode='min',
    baseline=None, restore_best_weights=True), epochs=50, batch_size = 137)

pred = model.predict(audio_test)

print(classification_report(Ytest,Thresholding(pred,0.8)))
# print(classification_report(Ytest,Thresholding(pred,0.6)))
# print(classification_report(Ytest,Thresholding(pred,0.4)))
# print(classification_report(Ytest,Thresholding(pred,0.3)))
# print(classification_report(Ytest,Thresholding(pred,0.7)))

--------------------------------------AUDIO (W/o GATING) SENTENCE LEVEL-------------------------------------------------
Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.6938 - val_loss: 0.6901
Epoch 2/50
1/1 [==============================] - 0s 358ms/step - loss: 0.6956 - val_loss: 0.6914
Epoch 3/50
1/1 [==============================] - 0s 370ms/step - loss: 0.6933 - val_loss: 0.6926
Epoch 4/50
1/1 [==============================] - 0s 356ms/step - loss: 0.6935 - val_loss: 0.6939
Epoch 5/50
1/1 [==============================] - 0s 353ms/step - loss: 0.6921 - val_loss: 0.6951
Epoch 6/50
1/1 [==============================] - 0s 374ms/step - loss: 0.6923 - val_loss: 0.6963
Epoch 7/50
1/1 [==============================] - 0s 355ms/step - loss: 0.6922 - val_loss: 0.6975
Epoch 8/50
1/1 [==============================] - 0s 373ms/step - loss: 0.6920 - val_loss: 0.6987
Epoch 9/50
1/1 [==============================] - 0s 353ms/step - loss: 0.6916 - val_loss: 0.7000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
print("--------------------------------------AUDIO + TEXT (W GATING) SENTENCE LEVEL-------------------------------------------------")
model1 = text_audio()
model = model1.run_model()

model.fit([audio_train,text_train],Ytrain, validation_split = 0.2, callbacks = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0, patience=15, verbose=0, mode='min',
        baseline=None, restore_best_weights=True),epochs=50, batch_size = 137)


from sklearn.metrics import classification_report
pred = model.predict([audio_test,text_test])
pred2 = model.predict([audio_train,text_train])

print(classification_report(Ytest,Thresholding(pred,0.7)))
# print(classification_report(Ytest,Thresholding(pred,0.6)))
# print(classification_report(Ytest,Thresholding(pred,0.4)))
# print(classification_report(Ytest,Thresholding(pred,0.3)))
# print(classification_report(Ytest,Thresholding(pred,0.7)))

--------------------------------------AUDIO + TEXT (W GATING) SENTENCE LEVEL-------------------------------------------------
Epoch 1/50
1/1 [==============================] - 10s 10s/step - loss: 0.0029 - val_loss: 0.0103
Epoch 2/50
1/1 [==============================] - 6s 6s/step - loss: 0.0028 - val_loss: 0.0104
Epoch 3/50
1/1 [==============================] - 4s 4s/step - loss: 0.0028 - val_loss: 0.0104
Epoch 4/50
1/1 [==============================] - 4s 4s/step - loss: 0.0028 - val_loss: 0.0104
Epoch 5/50
1/1 [==============================] - 4s 4s/step - loss: 0.0027 - val_loss: 0.0104
Epoch 6/50
1/1 [==============================] - 4s 4s/step - loss: 0.0027 - val_loss: 0.0105
Epoch 7/50
1/1 [==============================] - 4s 4s/step - loss: 0.0027 - val_loss: 0.0105
Epoch 8/50
1/1 [==============================] - 4s 4s/step - loss: 0.0027 - val_loss: 0.0105
Epoch 9/50
1/1 [==============================] - 4s 4s/step - loss: 0.0025 - val_loss: 0.0105
Epoch 10/50
1/1 [